In [ ]:
import pandas as pd

#steve-report is a recording of my shifts to-date. routes.csv is all routes from Google GTFS
driver_report = pd.read_csv('steve-report.csv')
routes = pd.read_csv('routes.csv')

In [ ]:
driver_report.route = driver_report['route'].astype('Int64').astype('str')
routes['route_id'] = routes['route_id'].astype('str')

In [ ]:
#Routes need to be changed to strip 39E of its character
#Regex: ( ) = capture group, \d = shorthand character class, same as [0-9], + = one or more expression
routes['route_id'] = routes['route_id'].str.extract('(\d+)', expand=False)

In [ ]:
#Working - do not touchhhh
df = pd.merge(driver_report, routes, left_on='route', right_on='route_id')
df.to_csv('results.csv')

In [ ]:
#get sum of hours for each route
df.length.astype('float64')
df_grp = df.groupby(['route_long_name', 'route_id'])['length'].sum().reset_index()

In [ ]:
#total hours spent driving per shift
import seaborn as sns
ax2 = sns.barplot(x="length", y="route_long_name", data=df_grp)

In [76]:
#Need to ignore errors because military time exceeds 23:59
#errors='ignore' gives just the time
#errors='coerce' gives the right output but also adds the day, which i don't want
df['start time'] = pd.to_datetime(df['start time'], errors='ignore')
df['end time'] = pd.to_datetime(df['end time'], errors='ignore')
df['date'] = pd.to_datetime(df['date'])

In [93]:
from datetime import datetime

#df['start time']

date_time_obj = datetime.strptime("16:10", '%H:%M')

date_time_obj

#This may be a workaround to the datetime issue

datetime.datetime(1900, 1, 1, 16, 10)

In [ ]:
#regex for shift_type
am_shifts = '^(.*)AM(.*)$'
mid_shifts = '^(.*)MID(.*)$'
pm_shifts = '^(.*)PM(.*)$'
pm2_shifts = '^(.*)PM2(.*)$'
eve_shifts = '^(.*)EVE(.*)$'

In [ ]:
#calculated column

df.loc[df['shiftname'].str.contains(am_shifts), 'shift_type'] = 'AM'
df.loc[df['shiftname'].str.contains(mid_shifts), 'shift_type'] = 'MID'
df.loc[df['shiftname'].str.contains(pm_shifts), 'shift_type'] = 'PM'
df.loc[df['shiftname'].str.contains(pm2_shifts), 'shift_type'] = 'PM2'
df.loc[df['shiftname'].str.contains(eve_shifts), 'shift_type'] = 'EVE'

In [ ]:
df[['date', 'start time']] = df[['date', 'start time']].apply(pd.to_datetime)

In [ ]:
df

In [ ]:
#Faceted area plot

import plotly.express as px

fig = px.bar(df, facet_col="route_long_name", facet_col_wrap=2, x=df.date, y=df.length, color=df.shift_type, height=1400, width=1200, title="<b>Bus Shifts To Date <i>(zoom to expand)</i></b>", labels=dict(length="Length (hours)", shift_type="Shift Type", route_long_name="Route"))
fig.update_layout(
    title_font_family="Minion Pro",
    paper_bgcolor="#881c1c",
    font=dict(
            family="Helvetica",
            size=20,
            color="white"
        )
)
fig.show()

df

In [ ]:
#3d scatter

import plotly.express as px

fig = px.scatter_3d(df, x=df['route_id'], y=df['date'], z=df['length'])
fig.show()

In [ ]:
fig = px.timeline(df, x_start="start time", x_end="end time", y="date")

fig.show()

In [ ]:
df['start time']

In [ ]:
df['end time']

In [ ]:
import folium

#Starts Folium centered on Amherst MA:
folium_map = folium.Map(location=[42.375968582346225, -72.520124958991],
                    zoom_start = 13) 
folium_map

In [ ]:
#The GEOJSON file has a "FeatureCollection" with "Features" inside of it.
#Each "feature" has a property equal to the bus route.
#Example: "properties": {"route_id": "30"}},

#Approaches:
#Add property for times driven to feature.property?
#Create new geojson files for later import into Leaflet/Folium? One per layer?
#Adjust script so that is done earlier?


In [ ]:
import json

route_shapes = './route_shapes.geojson'

folium.GeoJson(route_shapes, name="geojson").add_to(folium_map)

folium.LayerControl().add_to(folium_map)

folium_map


In [ ]:
from geojsplit import geojsplit

geojson = geojsplit.GeoJSONBatchStreamer("./route_shapes.geojson")

for features in geojson.stream():
    for feature in features:
        print('properties')